In [ ]:
from screener_client.fetch import fetch_all_data

BASE_URL = "https://www.screener.in/company/DIXON/consolidated/"

data = await fetch_all_data(BASE_URL)

In [ ]:
data["peers_api"]

In [ ]:
data["schedules"].keys()

In [ ]:
from db.db_utils import load_company_to_duckdb
load_company_to_duckdb(data, "db/screener_financials.duckdb")

In [ ]:
con = duckdb.connect("db/screener_financials.duckdb")

In [ ]:
data

In [ ]:
meta = con.execute("""
    SELECT * from financials_quarterly 
    """).fetchdf()
meta

In [ ]:
meta

In [ ]:
def build_company_llm_context(con, company_id: str) -> str:
    """
    Returns a text block with all relevant info for a given company_id.
    Missing pieces are simply skipped.
    """
    # 1) Basic info
    meta = con.execute("""
        SELECT c.company_id, c.company_name, c.source_url,
               s.market_cap_raw, s.current_price_raw, s.high_low_raw,
               s.stock_pe_raw, s.book_value_raw, s.dividend_yield_raw,
               s.roce_raw, s.roe_raw, s.face_value_raw
        FROM companies c
        LEFT JOIN company_summary s USING(company_id)
        WHERE c.company_id = ?
        """, [company_id]).fetchdf()

    # 2) Latest quarterly sales & profit (if present)
    latest_q = con.execute("""
        WITH latest_period AS (
            SELECT MAX(period) AS period
            FROM financials_quarterly
            WHERE company_id = ?
        )
        SELECT fq.item, fq.value
        FROM financials_quarterly fq
        JOIN latest_period lp
          ON fq.company_id = ? AND fq.period = lp.period
        """, [company_id, company_id]).fetchdf()

    # 3) Pros/cons
    analysis = con.execute("""
        SELECT pros, cons, about
        FROM analysis_insights
        WHERE company_id = ?
        """, [company_id]).fetchone()

    # Then compose a robust text summary
    lines = []

    if not meta.empty:
        m = meta.iloc[0]
        lines.append(f"Company: {m.company_name} (ID: {m.company_id})")
        if m.market_cap_raw:
            lines.append(f"Market cap: {m.market_cap_raw}")
        if m.current_price_raw:
            lines.append(f"Current price: {m.current_price_raw} (High/Low: {m.high_low_raw})")
        if m.stock_pe_raw:
            lines.append(f"Valuation: P/E {m.stock_pe_raw}, Book value {m.book_value_raw}, PBV from charts if available.")
        if m.roce_raw or m.roe_raw:
            lines.append(f"Returns: ROCE {m.roce_raw}, ROE {m.roe_raw}")

    if not latest_q.empty:
        # Example: pick some key items by name if they exist
        def get_item(df, name):
            row = df[df["item"] == name]
            return float(row["value"].iloc[0]) if not row.empty else None

        sales = get_item(latest_q, "Sales +")
        op_profit = get_item(latest_q, "Operating Profit")
        np = get_item(latest_q, "Net Profit +")
        opm = get_item(latest_q, "OPM %")

        lines.append("Latest quarterly snapshot:")
        if sales is not None:
            lines.append(f"- Sales: {sales}")
        if op_profit is not None:
            lines.append(f"- Operating profit: {op_profit}")
        if np is not None:
            lines.append(f"- Net profit: {np}")
        if opm is not None:
            lines.append(f"- OPM: {opm}%")

    if analysis is not None:
        pros_json, cons_json, about = analysis
        if about:
            lines.append("\nAbout:")
            lines.append(about)
        if pros_json:
            pros = pd.read_json(pros_json, typ="series").tolist()
            lines.append("\nPros:")
            for p in pros:
                lines.append(f"- {p}")
        if cons_json:
            cons = pd.read_json(cons_json, typ="series").tolist()
            lines.append("\nCons:")
            for c in cons:
                lines.append(f"- {c}")

    return "\n".join(lines)


In [ ]:
import duckdb
import pandas as pd

In [ ]:
con = duckdb.connect("db/screener_financials.duckdb")

In [ ]:
context = build_company_llm_context(con, company_id="3467")

prompt = f"""
You are an equity research assistant.

Analyze the following company in detail. Highlight growth, profitability, capital efficiency, leverage,
and any key risks.

{context}
"""

In [ ]:
print(context)